In [19]:
#Import needed libraries
import random

#Define the Deck
def deck_initialize():
    hearts = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    deck = hearts*4
    return deck

#Define the values for each card of the deck
def deck_values():
    deck_values = {}
    hearts = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    for card in hearts:
        if card in ['J','Q','K']:
            deck_values[card] = 10
        elif card == 'A':
            deck_values[card] = 11
        else:
            deck_values[card] = int(card)
    return(deck_values)    
        
# The player has a total amount of money (pot), defined at the begining of the game. The minimum bet is 5, thus the minimum pot is also 5.
def money_on_the_table():
    pot = 0
    while pot < 5:
        pot = int(input("How much money do you want to bet on this table. The minimum bet is $5\n"))
    return(pot)

#The player places the bet at the beginning of each round
def bet_placing():
    bet = 0
    while (bet < 5) or (bet > 200):
        bet = int(input("How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5\n"))
        if bet < 5:
            print("The minimum bet is $5")
        elif bet > 200:
            print("The maximum bet is $200")
        else:
            return(bet)
        
#The dealer deals the initial hands for a given deck. Retuns a tuple(?) with 2 lists, 2 items each, the player's hand and the dealer's hand
#WARNING when printing the result, the second card of the dealer's hand should not be shown
def deal_cards(deck):
    player_hand = [random.choice(deck) for x in range(2)]
    dealer_hand = [random.choice(deck) for x in range(2)]
    return player_hand, dealer_hand

#The player chooses what to do
def player_action():
    options = ["hit","pass"]
    player_choice = ""
    while player_choice not in options:
        player_choice = str(input("You can hit or pass, what would you like to do?\n"))
    return player_choice

#We calculate the sum of a hand. Needs a list and a dictionary
def hand_sum(hand,values):
    total = 0
    for card in hand:
        total += values[card]
    return total

#We ask if the player wants to keep playing
def keep_playing():
    if pot >= 5:
        return 'Y' == input("Do you want to keep playing Y/N")
    else:
        print("You don't have enough chips on your pot :(")
        return False
    
#Build the game:
#Ask the player how much money he wants to put in the pot
pot = money_on_the_table()
print(f"You currently have ${pot}")

#We create a loop for each round
give_me_more = True
while give_me_more:
    #We create the deck
    deck = deck_initialize()

    #We ask the player to place a bet for the current round
    bet = bet_placing()

    #We deal the cards and display the current status
    print("Dealing...\n...\n...")
    player_hand = deal_cards(deck)[0]
    dealer_hand = deal_cards(deck)[1]
    print("Your hand is:",player_hand[0],player_hand[1])
    print("The dealer's hand is:",dealer_hand[0],"[X]")

    #We give the player the choice to take action
    print("What would you like to do?")

    while player_action() == "hit":
        player_hand.append(random.choice(deck))
    #PROBLEM: I'M TAKING AS MANY CARDS AS I WANT, NOT LIMITED TO ONE DECK. ACTUALLY ONE DECK IS MORE RESTRICTIVE THAN "INFINITE" DECKS. IF I CHOOSE TO HAVE INFINITE DECKS, I DON'T NEEC TO INITIALIZE THE DECK BY MULTIPLYING BY 4
        if hand_sum(player_hand,deck_values()) > 21:
            print(player_hand,"\nYou are now at",hand_sum(player_hand,deck_values()),"\nBusted!")
            pot = pot - bet
        else:
            print("Your hand is:",player_hand,"\nYou are now at",hand_sum(player_hand,deck_values()))

    print("Your hand is",player_hand,"You are now at",hand_sum(player_hand,deck_values()))
    #player_action()==pass. We now have either the player busted or with a hand and the total value displayed

    #Dealer's play
    while hand_sum(dealer_hand,deck_values()) < 17:
        if hand_sum(dealer_hand,deck_values()) > 21:
            print("Dealer is busted! You win")
            pot = pot + bet
        dealer_hand.append(random.choice(deck))

    print("The dealer's hand is:",dealer_hand,"\nWith a total score of",hand_sum(dealer_hand,deck_values()))

    #We settle the game
    if hand_sum(dealer_hand,deck_values()) == 21 and hand_sum(player_hand,deck_values()) == 21:
        if 'A' in dealer_hand not in player_hand:
            pot = pot - bet
            print(f"The dealer wins\n Your remaining pot is ${pot}")
        elif 'A' in player_hand not in dealer_hand:
            pot = pot + bet
            print(f"You win!\n You have now ${pot} in your pot")
        else:
            print("It's a split")
    else:
        if hand_sum(dealer_hand,deck_values()) > hand_sum(player_hand,deck_values()):
            pot = pot - bet
            print(f"The dealer wins\n Your remaining pot is ${pot}")
        elif hand_sum(dealer_hand,deck_values()) < hand_sum(player_hand,deck_values()):
            pot = pot + bet
            print(f"You win!\n You have now ${pot} in your pot")
        else:
            print("It's a split")

    give_me_more = keep_playing()

How much money do you want to bet on this table. The minimum bet is $5
20
You currently have $20
How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5
10
Dealing...
...
...
Your hand is: 10 10
The dealer's hand is: Q [X]
What would you like to do?
You can hit or pass, what would you like to do?
pass
Your hand is ['10', '10'] You are now at 20
The dealer's hand is: ['Q', 'A'] 
With a total score of 21
The dealer wins
 Your remaining pot is $10
Do you want to keep playing Y/NY
How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5
10
Dealing...
...
...
Your hand is: 8 9
The dealer's hand is: 8 [X]
What would you like to do?
You can hit or pass, what would you like to do?
pass
Your hand is ['8', '9'] You are now at 17
The dealer's hand is: ['8', 'A'] 
With a total score of 19
The dealer wins
 Your remaining pot is $0
You don't have enough chips on your pot :(
